### Neuronal system simulation

The following code will simulate the behaviour of the 90 regions of interest, only concerning itself with the neuronal system. The data from this simulation can be transformed into artificial BOLD fMRI data to be used to extract leading eigenvectors and form a PMS. The code in this block was adapted from optim_placebo_psilo.m from the original paper's repository. 

The output should be the global coupling parameter G=1.6. Remember that this system is attempting to minimize the KLD between it's generated PMS and the original impirical PMS. 

Issues to resolve:
-TR is has been arbitraily defined to be 3, I don't know what it really is, maybe number of groups in PMS?

In [19]:
import numpy
from utils import Balance_J
from utils import phii
from utils import phie

#there is a large segment of code that I cannot interpret at this point, so I have skipped it for now
#I will focus on writing the rest of the code, appearing to involve the actual simulation.

#importing functions

#defining parameters
dtt= 1e-3 #sampling rate of the simulated neuronal activity
dt=0.1

taon=100
taog=10
gamma=0.641
sigma=0.01
JN=0.15
I0=0.382
Jexte=1.
Jexti=0.7
w=1.4
TR=3
Tmax=2000
boldstep=TR*1000

WE=numpy.arange(0.5,2,0.01).tolist() #list from 0.5 -> 2 with a step of 0.01
C=10 #there is another definition for C, but I don't know how it works

for we in WE:
    
    Cnew=C

    print("Running Balance_J, iteration:", we)
    J=Balance_J(we,Cnew) #this definition of Cnew relies on the incorrect definition of C
    print("Finished Balance_J!")

    for i in range(10):
        neuro_act=numpy.zeros((round((1000*(Tmax-1)*TR+1)),C)) #this doesn't make any sense, TR is a parameter undefined at this point, might be equal to 3, described as the repition time in seconds
        sn=0.001*(numpy.ones((C,1)))
        sg=0.001*(numpy.ones((C,1)))
        nn=1
        for t in range(int((1000*(Tmax-1)*TR)/dt)):
            xn=I0*Jexte+w*JN*sn+we*JN*Cnew*sn-J*sg
            xg=I0*Jexti+JN*sn-sg
            rn=phie(xn)
            rg=phii(xg)
            noise=numpy.random.randn(C,1)
            sn=sn+dt*(-sn/taon+(1-sn)*gamma*rn/1000)+sqrt(dt)*sigma*noise
            for region in sn: #get this checked to make sure it is inline with the matlab code, original code was: sn(sn>1) = 1 (newline) sn(sn<0) = 0
                if region>1:
                    region=1
                elif region<0:
                    region=0
            noise=numpy.random.randn(C,1)
            sg=sg+dt*(-sg/taog+rg/1000)+sqrt(dt)*sigma*noise
            for region in sg: #get this checked to make sure it is inline with the matlab code, original code was: sn(sn>1) = 1 (newline) sn(sn<0) = 0
                if region>1:
                    region=1
                elif region<0:
                    region=0

            print("sn[0] after iteration", t,"=", sn[0])
            print("sg[0] after iteration", t, "=", sg[0])

            j=j+1
            if abs(t%1)<0.01: #this should definetly get checked
                neuro_act[nn]=rn
                nn=nn+1
        

return(sn,sg)

Running Balance_J, iteration: 0.5
0
1
2
3
4
5
6
7
8
9
Finished Balance_J!


NameError: name 'sqrt' is not defined